# Package Imports/Boiler Plate

In [101]:
# install packages into jupyter runtime if not already installed
!pip install uv --user
!uv pip install --system cloud-volume
!uv pip install --system navis
!uv pip install --system networkx
!uv pip install --system matplotlib

Using Python 3.11.11 environment at /usr
Audited 1 package in 108ms
Using Python 3.11.11 environment at /usr
Audited 1 package in 107ms
Using Python 3.11.11 environment at /usr
Audited 1 package in 104ms
Using Python 3.11.11 environment at /usr
Audited 1 package in 96ms


In [102]:
# only run if in colab; not local runtime
import google.colab.auth
google.colab.auth.authenticate_user()

In [103]:
import cloudvolume
from cloudvolume import CloudVolume
import os
import logging
import navis
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict

# Importing the Neuron Itself


In [104]:
# id or neuron to run through pipeline with
# e.g. 3896803064 in h01
neuron_id = 3896803064

In [105]:
# path of cloudvolume bucket
bucket_path = 'h01-release/data/20210601/c3/'

In [106]:
# load in the bucket using cloudvolume
cv = CloudVolume(f'gs://{bucket_path}', mip=0, cache=False, use_https=True)
print(cv.info)

{'@type': 'neuroglancer_multiscale_volume', 'data_type': 'uint64', 'num_channels': 1, 'scales': [{'chunk_sizes': [[128, 128, 32]], 'compressed_segmentation_block_size': [16, 16, 4], 'encoding': 'compressed_segmentation', 'key': '8.0x8.0x33.0', 'resolution': [8, 8, 33], 'sharding': {'@type': 'neuroglancer_uint64_sharded_v1', 'data_encoding': 'gzip', 'hash': 'identity', 'minishard_bits': 6, 'minishard_index_encoding': 'gzip', 'preshift_bits': 9, 'shard_bits': 17}, 'size': [515892, 356400, 5293]}, {'chunk_sizes': [[128, 128, 64]], 'compressed_segmentation_block_size': [8, 8, 4], 'encoding': 'compressed_segmentation', 'key': '16.0x16.0x33.0', 'resolution': [16, 16, 33], 'sharding': {'@type': 'neuroglancer_uint64_sharded_v1', 'data_encoding': 'gzip', 'hash': 'identity', 'minishard_bits': 6, 'minishard_index_encoding': 'gzip', 'preshift_bits': 9, 'shard_bits': 14}, 'size': [257946, 178200, 5293]}, {'chunk_sizes': [[64, 64, 64]], 'compressed_segmentation_block_size': [8, 8, 8], 'encoding': 'c

# Extracting and Healing the Skeleton

In [107]:
# extract the neuron skeleton corresponding to the neuron_id
skeletons = cv.skeleton.get([neuron_id])
skeleton = skeletons[0] if skeletons else None

# delete old swc
if os.path.exists(f'{neuron_id}.swc'):
  os.remove(f'{neuron_id}.swc')

# save the neuron skeleton to swc
swc_file = f'{neuron_id}.swc'
with open(swc_file, 'w') as f:
    f.write(skeleton.to_swc())
    print(f'Skeleton saved as SWC: {swc_file}')

Decompressing: 100%|██████████| 1/1 [00:00<00:00, 177.01it/s]


Skeleton saved as SWC: 3896803064.swc


In [108]:
# load the skeleton into a navis.TreeNeuron object for further processing
neuron = navis.read_swc(swc_file)

if type(neuron) != navis.TreeNeuron:
  logging.error(f'Neuron type is {type(neuron)} and not a navis.TreeNeuron')

print(neuron)
print(neuron.nodes)

type                      navis.TreeNeuron
name                            3896803064
n_nodes                              50521
n_connectors                          None
n_branches                            3841
n_leafs                               3898
cable_length                    15344019.0
soma                                  None
units                      1 dimensionless
created_at      2025-02-08 20:59:35.693374
origin                      3896803064.swc
file                        3896803064.swc
dtype: object
       node_id label          x          y         z      radius  parent_id  \
0            1     0  2598016.0  1305120.0  142197.0   96.000000         -1   
1            2     0  2598016.0  1305440.0  142164.0  106.437775          1   
2            3     0  2598016.0  1305728.0  142032.0  172.325272          2   
3            4     0  2598048.0  1305888.0  141735.0   96.337944          3   
4            5     0  2598144.0  1306016.0  141471.0   45.967381          4

In [109]:
# fix holes and missing connections by healing the neuron
navis.heal_skeleton(neuron, inplace=True)

,
type,navis.TreeNeuron
name,3896803064
n_nodes,50521
n_connectors,None
n_branches,3884
n_leafs,3902
cable_length,15406663.0
soma,None
units,1 dimensionless
created_at,2025-02-08 20:59:35.693374


In [110]:
# examples of skeleton vertices
neuron.nodes[['x', 'y', 'z']].values.tolist()[:10]

[[2598016.0, 1305120.0, 142197.0],
 [2598016.0, 1305440.0, 142164.0],
 [2598016.0, 1305728.0, 142032.0],
 [2598048.0, 1305888.0, 141735.0],
 [2598144.0, 1306016.0, 141471.0],
 [2598368.0, 1306048.0, 141273.0],
 [2598656.0, 1306144.0, 141240.0],
 [2598944.0, 1306208.0, 141207.0],
 [2599040.0, 1306080.0, 141471.0],
 [2599040.0, 1305888.0, 141702.0]]

In [111]:
neuron_coords = neuron.nodes[['x', 'y', 'z']].values.tolist()

In [112]:
# save neuron skeleton vertices as a numpy array (optional; for testing outside of notebook)
np.save(f'skeleton_vert_{neuron_id}.npy', neuron_coords)

# Generating Synapse Point Cloud

In [113]:
# materialization table from SynAnno
# e.g. '/content/synapse-export_000000000000.csv' from a local runtime
materialization_url = '/content/synapse-export_000000000000.csv'

In [114]:
# read in the materialization table into a pandas dataframe
try:
  synapse_df = pd.read_csv('/content/synapse-export_000000000000.csv')
except FileNotFoundError:
  print("Error: synapse-export_000000000000.csv not found. Please check the file path.")

  exit()


# specifically single out rows with the neuron_id at hand
filtered_df = synapse_df[
    (synapse_df['pre_neuron_id'] == neuron_id) | (synapse_df['post_neuron_id'] == neuron_id)
]

filtered_df


,pre_pt_x,pre_pt_y,pre_pt_z,post_pt_x,post_pt_y,post_pt_z,x,y,z,pre_neuron_id,post_neuron_id
34750,294170,175882,265,294167,175870,265,294168,175875,265,4768162304,3896803064
58677,323545,169154,2063,323553,169162,2063,323550,169159,2063,39881114715,3896803064
73175,303140,180671,1299,303131,180665,1298,303135,180670,1299,3896803064,22586492020
93032,325864,169910,2584,325857,169899,2584,325862,169906,2584,48761076801,3896803064
133923,328671,167807,1059,328678,167796,1059,328674,167802,1059,22179722775,3896803064
160738,309797,185663,1833,309811,185668,1834,309805,185666,1834,31612174300,3896803064
162464,364850,172870,4138,364842,172864,4138,364846,172868,4138,66740456744,3896803064
207325,346207,162450,4918,346210,162438,4918,346210,162438,4918,-1,3896803064
230864,301640,176232,829,301630,176237,829,301636,176233,829,4753882153,3896803064
230948,364482,142461,1085,364473,142467,1085,364478,142462,1085,3678766702,3896803064


In [115]:
# convert synapse centroids into a point cloud
# scale points by 8,8,33 to reflect voxel-nanometer conversion
try:
  point_cloud = np.column_stack((filtered_df['x']*8, filtered_df['y']*8, filtered_df['z']*33))

  # print a small set of synapse coordinates
  # should be similar to skel verts from end of prev section
  print(point_cloud[:10])
except KeyError as e:
    print(f"Error: Column '{e}' not found in the DataFrame. Please check the column names.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


[[2353344 1407000    8745]
 [2588400 1353272   68079]
 [2425080 1445360   42867]
 [2606896 1359248   85272]
 [2629392 1342416   34947]
 [2478440 1485328   60522]
 [2918768 1382944  136554]
 [2769680 1299504  162294]
 [2413088 1409864   27357]
 [2915824 1139696   35805]]


In [116]:
# save synapse point cloud as a numpy array (optional; for testing outside of notebook)
np.save(f'synapse_point_cloud_{neuron_id}.npy', point_cloud)

### Snap Synapse Points to Neurons

In [117]:
# convert the synapse point cloud into a kdtree and snap via that
from scipy.spatial import KDTree

scaled_point_cloud = np.array(point_cloud)
neuron_coords = np.array(neuron_coords)

neuron_tree = KDTree(neuron_coords)

_, indices = neuron_tree.query(scaled_point_cloud)

snapped_points = neuron_coords[indices]

In [118]:
# see where the synapses snapped
for i in range(len(scaled_point_cloud)):
    print(f"Original: {scaled_point_cloud[i]}, Snapped: {snapped_points[i]}")

Original: [2353344 1407000    8745], Snapped: [2353312. 1406880.    8481.]
Original: [2588400 1353272   68079], Snapped: [2588640. 1353824.   66957.]
Original: [2425080 1445360   42867], Snapped: [2425184. 1445312.   42966.]
Original: [2606896 1359248   85272], Snapped: [2606176. 1358752.   87054.]
Original: [2629392 1342416   34947], Snapped: [2629440. 1342208.   34749.]
Original: [2478440 1485328   60522], Snapped: [2478912. 1485536.   60621.]
Original: [2918768 1382944  136554], Snapped: [2918624. 1382784.  136587.]
Original: [2769680 1299504  162294], Snapped: [2769632. 1299488.  162558.]
Original: [2413088 1409864   27357], Snapped: [2412928. 1409888.   27555.]
Original: [2915824 1139696   35805], Snapped: [2915712. 1139808.   35739.]
Original: [2407488 1390120   78078], Snapped: [2407136. 1390912.   77814.]
Original: [2375416 1423384   19008], Snapped: [2375200. 1423232.   19140.]
Original: [2471288 1440480   35904], Snapped: [2471072. 1439936.   34122.]
Original: [2639200 134048

In [121]:
# save snapped synapse point cloud as a numpy array
np.save(f'snapped_synapse_point_cloud_{neuron_id}.npy', snapped_points)

### Test Plots

In [119]:
# plot of the scaled synapse cloud and the skeleton

# create the Plotly figure object
fig = go.Figure()

# add the neuron nodes/verts to the plot
fig.add_trace(go.Scatter3d(
    x=neuron.nodes['x'],
    y=neuron.nodes['y'],
    z=neuron.nodes['z'],
    mode='markers',
    marker=dict(size=3, color='blue'),
    name='Neuron Nodes'
))

# add the scaled synapse point cloud to the plot
fig.add_trace(go.Scatter3d(
    x=point_cloud[:, 0],
    y=point_cloud[:, 1],
    z=point_cloud[:, 2],
    mode='markers',
    marker=dict(size=5, color='red'),
    name='Scaled Synapse Points'
))

fig.update_layout(
    title='Neuron Nodes and Scaled Point Cloud',
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z'
    )
)

# show the plot!
fig.show()


In [100]:
# plot of the scaled synapse cloud and the skeleton

# create the Plotly figure object
fig = go.Figure()

# add the neuron nodes/verts to the plot
fig.add_trace(go.Scatter3d(
    x=neuron_coords[:, 0],
    y=neuron_coords[:, 1],
    z=neuron_coords[:, 2],
    mode='markers',
    marker=dict(size=3, color='blue', opacity=0.2),
    name='Neuron Skeleton'
))

# add the scaled synapse point cloud to the plot
fig.add_trace(go.Scatter3d(
    x=scaled_point_cloud[:, 0],
    y=scaled_point_cloud[:, 1],
    z=scaled_point_cloud[:, 2],
    mode='markers',
    marker=dict(size=5, color='red'),
    name='Original Synapse Points'
))

# add the snapped synapse point cloud to the plot
fig.add_trace(go.Scatter3d(
    x=snapped_points[:, 0],
    y=snapped_points[:, 1],
    z=snapped_points[:, 2],
    mode='markers',
    marker=dict(size=5, color='yellow'),
    name='Snapped Synapse Points'
))

fig.update_layout(
    title='Neuron, Synapse Points, and Snapped Points',
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z'
    )
)

# show the plot!
fig.show()